In [ ]:
import matplotlib.pyplot as plt
import scipy as sp
import networkx as nx
import numpy as np
import math
import time
import copy
import math
import random

from scipy.io import mmread
from scipy.sparse import csr_matrix
from networkx.utils import reverse_cuthill_mckee_ordering #RCM

# DESCRIÇÃO - Função para receber matriz .mtx
# SAÍDA - MATRIZ
def load_matriz(name_mtz): 
    matriz = open(name_mtz, "r")
    return matriz

# DESCRIÇÃO - Funcao para retornar o maximo da largura de banda - é chamada antes e depois para calcular o valor da matriz inicial e final
# ENTRADA - Recebe como parâmetro uma matriz de adjacencia
# SAÍDA - Retorna o valor maior da largura de banda
def bandwidth_max(matadj): 
    vx = matadj.shape[0]
    vy = matadj.shape[1]
    maxim = 1
    for i in range(vx):
        for j in range(vy):
            if matadj[i,j] > 0 and matadj[i,j] > maxim:
                maxim = matadj[i,j]                
    return maxim 

# DESCRIÇÃO - verifica o peso das arestas
# ENTRADA - Recebe como parâmetro uma matriz de adjacencia csr
# SAIDA Retorna matriz de adjacencia com pesos
def checks_edges(matadj_csr): 
    vx = matadj_csr.shape[0]
    vy = matadj_csr.shape[1]
    for i in range(vx):
        for j in range(vy):
            if (matadj_csr[i,j] != 0):
                if (j - i > 0):
                    matadj_csr[i,j] = j - i
                else:
                    matadj_csr[i,j] = -(j - i)
    return matadj_csr 

# DESCRIÇÃO - Função de menor grau utilizada para auxiliar na função do RCM
# ENTRADA - Recebe um grafo
# SAÍDA - Retorna o vertice de menor grau
def smallest_degree(G):
    return min(G, key=G.degree)
# Funcao para escolher um vertice aleatorio de um grafo

# DESCRIÇÃO - Função de para seleciona um vertice aleatorio utilizada para auxiliar na função do RCM
# ENTRADA - Recebe um grafo
# SAÍDA - Retorna o vertice aleatorio
def random_v(g): # Recebe um grafo
    # Retorna um vertice aleatorio
    return random.randint(0,(g.number_of_nodes()-1))

# DESCRIÇÃO - Funcao para aplicar a heuristica reverse_cuthill_mckee
# ENTRADA - Recebe um grafo
# SAÍDA - Retorna uma matriz esparsa com os pesos atualizados depois de aplicar o rcm
def rcm(graph): 
    list_rcm = list(reverse_cuthill_mckee_ordering(graph,heuristic=smallest_degree))
    return checks_edges(nx.to_scipy_sparse_matrix(graph, nodelist=list_rcm))

# DESCRIÇÃO - Funcao para aplicar a heuristica reverse_cuthill_mckee
# ENTRADA - Recebe um grafo
# SAÍDA - Retorna uma matriz esparsa com os pesos atualizados depois de aplicar o rcm
def rcm_small(graph): 
    list_rcm = list(reverse_cuthill_mckee_ordering(graph,heuristic=smallest_degree))
    return checks_edges(nx.to_scipy_sparse_matrix(graph, nodelist=list_rcm))

# DESCRIÇÃO - Utiliza scipy_sparse_matrix para transformar a matriz em um grafo
# ENTRADA -  Matriz
# SAÍDA - GRAFO
def trans_graph(matriz):
    g = nx.from_scipy_sparse_matrix(matriz)
    return g

# DESCRIÇÃO - Função para gerar a população inicial
# ENTRADA - Recebe uma matriz de adjacencia e o tamanho
# SAÍDA - Retorna uma lista de matrizes
# VARIAVEIS - pop - população, s - tamanho        
def generate_p_init(matadj, s):                          
    gp = nx.from_scipy_sparse_matrix(matadj)
    pop = []

    for i in range(0, s):
        pop.append(rcm(gp))
    
    return pop 

# DESCRIÇÃO - Função para cruzar dois membros da população
# ENTRADA - listas de lb da população, de matrizes da população, dois indices a serem cruzados e a tx de mutação
# SAÍDA - Retorna a matriz (membro) que possui a menor largura de banda entre as duas
def crossover(index1, index2, pop, list_bws,tx_mutation): 
    x = random.uniform(0.00, 1)

    if (list_bws[index1] > list_bws[index2]):
        if (x <= tx_mutation):
            return pop[index1]
        return generate_new_member(pop[index2]) 
    else:
        if (x <= tx_mutation): 
            return pop[index2]
        return generate_new_member(pop[index1])
    
# DESCRIÇÃO - Função para gerar um novo membro
# ENTRADA - matriz
# SAÍDA - Retorna uma matriz que representa o membro
def generate_new_member(matriz): 
    return rcm(nx.from_scipy_sparse_matrix(matriz))    

# DESCRIÇÃO - Função para calcular a largura de banda de cada membro da populacao
# ENTRADA - Recebe a lista (pupulacao) de matrizes
# SAÍDA - lista das lw de todos os membros da população
def checks_bandwidth_pop(pop): 
    bwp = []
    for i in range(0, len(pop)):
        bwp.append(bandwidth_max(pop[i]))
    
    return bwp    

# DESCRIÇÃO - Função para selecionar um membro
# ENTRADA - Recebe a lista de largura da 
# SAÍDA - lista das lw de todos os membros da população
def select_breeder(list_bands): 
    array = np.array(list_bands)
    temp = array.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(array))

    fitness = [len(ranks) - x for x in ranks]    
    cum_scores = copy.deepcopy(fitness)
    
    for i in range(1,len(cum_scores)):
        cum_scores[i] = fitness[i] + cum_scores[i-1]
        
    probs = [x / cum_scores[-1] for x in cum_scores]    
    rand = random.random()
    
    for i in range(0, len(probs)):
        if rand < probs[i]:
            return i
        
def represetation_grafhic(bandwidth,m_history):
    v1 = []
    v2 = []
    bandwidth = bandwidth + 100
    vx = m_history.shape[0]
    vy = m_history.shape[1]

    for i in range(vx):
        for j in range(vy):
            if (m_history[i,j] != 0):
                v1.append(i)
                v2.append(j)

    plt.plot(v1,v2,'ro')
    plt.axis([-10, bandwidth, -10, bandwidth])
    plt.show()

if __name__ == '__main__':
    name_mtz = ['mycielskian3.mtx','bcspwr02.mtx','iris_dataset_30NN.mtx','Trefethen_200.mtx','494_bus.mtx','662_bus.mtx']
    s_pop = 20
    n_pairs = 8
    n_iterations = 20
    pbb_m = 0.05 
    t_init_total = time.time() # Tempo total do no inicio
    for i_m in range(len(name_mtz)):        
        print("MATRIZ - ", name_mtz[i_m])        
        representacao = 0
        count = 0
               
        print("Execução\t Largura de Banda Inicial \t Largura de Banda Final \t\t\t Tempo")
        while count <= 11:
            t_init = time.time()

            representacao=count+1
            matriz = checks_edges(sp.io.mmread(load_matriz(name_mtz[i_m])).tocsr()) 
            bandwidth_init = bandwidth_max(matriz)

            #represetation_grafhic(bandwidth_init,matriz)

            pop = generate_p_init(matriz, s_pop)
            m_history = matriz
            vm_history = matriz.shape[0]
            for i in range(0, n_iterations):
                new_pop = []

                # Calcula a largura de banda para todos os membros da populacao
                bw_pop = checks_bandwidth_pop(pop)

                # Menor valor de largura de banda
                vM = min(bw_pop)

                # Indice da menor largura de banda
                im = np.argmin(bw_pop)

                if (vM < vm_history):
                    vm_history = vM
                    m_history = pop[im]

                # Cria a nova populacao cruzando os melhores individuos
                for j in range(0, n_pairs):
                    new_member = crossover(select_breeder(bw_pop),select_breeder(bw_pop), pop, bw_pop,pbb_m)
                    new_pop.append(new_member)

                # Adiciona novos membros aleatorios
                while (len(new_pop) < s_pop):
                    new_pop.append(generate_new_member(matriz))

                # Copia a populacao antiga pela populacao nova
                pop = copy.deepcopy(new_pop)
            print("%d \t\t\t\t %d \t\t\t\t %d  \t\t\t\t %f s\t\t\t\t " % (representacao, bandwidth_init,
                                                                          vm_history, time.time() - t_init))
            
            matriz = []      
            bandwidth_init = []
            vm_history = []
            t_init = 0
            count += 1
        print("===========================================================================================================\n")
    print("Tempo total de todas as execuções:", time.time()-t_init_total, "s")    
        #represetation_grafhic(bandwidth_init,m_history)



MATRIZ -  mycielskian3.mtx
Execução	 Largura de Banda Inicial 	 Largura de Banda Final 			 Tempo
1 				 3 				 2  				 1.724586 s				 
2 				 3 				 2  				 1.637510 s				 
3 				 3 				 2  				 2.095367 s				 
4 				 3 				 2  				 2.104089 s				 
5 				 3 				 2  				 2.271419 s				 
6 				 3 				 2  				 2.041719 s				 
7 				 3 				 2  				 1.946142 s				 
8 				 3 				 2  				 1.897500 s				 
9 				 3 				 2  				 1.936269 s				 
10 				 3 				 2  				 1.866624 s				 
11 				 3 				 2  				 2.167222 s				 
12 				 3 				 2  				 1.899731 s				 

MATRIZ -  bcspwr02.mtx
Execução	 Largura de Banda Inicial 	 Largura de Banda Final 			 Tempo
1 				 34 				 13  				 91.965484 s				 
2 				 34 				 13  				 96.077967 s				 
3 				 34 				 13  				 95.395076 s				 
4 				 34 				 13  				 105.533019 s				 
5 				 34 				 13  				 96.631844 s				 
6 				 34 				 13  				 95.268563 s				 
7 				 34 				 13  				 96.634697 s				 
8 				 34 				 13  				 96.704645 s				 
9 				 34 				 13  		